<a href="https://colab.research.google.com/github/kromeo6/chatbot/blob/main/chatbot_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import files
import pandas as pd
import numpy as np

In [5]:
!python -V

Python 3.6.9


In [7]:
uploaded = files.upload()

Saving intents1.json to intents1 (1).json


In [8]:
import io
data = pd.read_json(io.StringIO(uploaded['intents1.json'].decode('utf-8')))   #reading json file

In [31]:
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [9]:
import nltk
import numpy as np
import json
#import tensorflow
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
geo_words_mapping = {'ასაკი':['ასაკმა','ასაკს','ასაკის','ასასაკით','ასაკად', 'ასაკში', 'ასაკთან', 'ასაკზე'],
 'გამარჯობა':['გამარჯობათ','გამარჯობად','გაგიმარჯოს','მოგესალმებით','სალამი','პრივეტ','ჩაო'],
 'გქვია':['გქვიათ','დაგარქვა','დაგარქვათ','შეგარქვათ'],
 'დაგიძახო':['დაგიძახოთ','დაძახება','გეძახდით','გეძახდათ','გეძახდა'],
 'კარგად':['კარგათ', 'კარგი'],
 'მადლობა':['მადლობთ','გმადლობთ','მადლობელი', 'მადლიერი','მადლობ'],
 'მუშაობა':['მუშაობთ','მუშაობ','სამუშაო','სამუშავო','იმუშავებთ','მუშაობდით','მუშაობდი'],
 'ნახვამდის':['ნახვამდის'],
 'რამდენი':['რამდენში', 'რამდენად','რამდენზე','რამდენით'],
 'როგორ':['რანაირად','რანაირათ'],
 'რომელი':['რომელ','რომელიმე','რომელმა','რომელს','რომელის','რომელით'],
 'საათი':['საატი','საათამდე','საატამდე','საათზე','საატზე','საათში','სათამდე','საათით','საათები'],
 'სახელი':['სახელად','სახელათ','სახელში','სახელმა','სახელს','სახელით','სახელთან'],
 'წელი':['წლის','წელმა','წელს','წლით','წლად','წელო','წელში','წელთან'],
 'ხარ':['ხართ']
 }

In [11]:
def replace_words(l, d):  
  p = []
  for w in l:
    lnth = len(p)
    for k in d:
      if w in d[k]:
        p.append(k)
        continue
    if len(p) == lnth:
      p.append(w)   
  return p    

In [18]:
def get_training_data_from_json (data, d=0, replace=True):
        
    words = [] #we need to put all the used word in patterns into this list
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern) #dissolve sentance into words            
            words.extend(wrds)
            docs_x.append(wrds) #start to make training data
            docs_y.append(intent["tag"]) #start to make the target

        if intent["tag"] not in labels:
            labels.append(intent["tag"]) #labels are set of tags
    
    words = sorted(list(set(words)))  # set function gives us the unique words
    labels = sorted(labels)
    if replace:
      docs_x_replaced = []
      words = replace_words(words, d)
      for doc in docs_x:
        docs_x_replaced.append(replace_words(doc ,d))
      return docs_x_replaced, docs_y, words, labels
    return docs_x, docs_y, words, labels  

In [13]:
def to_vectors(docs_x,docs_y, words,labels):
  training = []
  output = []
  out_empty = [0 for _ in range(len(labels))]

  # we need to convert word into vectors. we created list of all unique words into "words" list.

  for x, doc in enumerate(docs_x):
      bag = []

      #wrds = [stemmer.stem(w.lower()) for w in doc]
      wrds = [w for w in doc]  # es sworia mara zedmetia

      for w in words:
          if w in wrds:
              bag.append(1)
          else:
              bag.append(0)

      output_row = out_empty[:]
      output_row[labels.index(docs_y[x])] = 1

      training.append(bag)
      output.append(output_row)
      training = np.array(training)
      output = np.array(output)
      return training, output

In [19]:
docs_x, docs_y, words, labels = get_training_data_from_json (data, d=geo_words_mapping)

In [20]:
training, output = to_vectors(docs_x,docs_y, words,labels)

In [38]:
#words = [stemmer.stem(w.lower()) for w in words if w != "?"] #all the letters should be NOT capital

In [115]:
# my_dict = json.dumps(d, ensure_ascii=False)
# print(my_dict)

In [1]:
%tensorflow_version 1.x


TensorFlow 1.x selected.


In [2]:
import tflearn
import tensorflow

In [21]:
tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
keep_dims is deprecated, use keepdims instead





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where







In [22]:
model.fit(training, output, n_epoch=100, batch_size=8, show_metric=True)
model.save("model.tflearn")

---------------------------------
Run id: HMB4P7
Log directory: /tmp/tflearn_logs/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.
---------------------------------
Training samples: 1
Validation samples: 0
--
Training Step: 1  | time: 0.321s
| Adam | epoch: 001 | loss: 0.00000 - acc: 0.0000 -- iter: 1/1
--
Training Step: 2  | total loss: 1.44851 | time: 0.006s
| Adam | epoch: 002 | loss: 1.44851 - acc: 0.0000 -- iter: 1/1
--
Training Step: 3  | total loss: 1.57878 | time: 0.004s
| Adam | epoch: 003 | loss: 1.57878 - acc: 0.8182 -- iter: 1/1
--
Training Step: 4  | total loss: 1.59919 | time: 0.005s
| Adam | epoch: 004 | loss: 1.59919 - acc: 0.9545 -- iter: 1/1
--
Training Step: 5  | total loss: 1.60268 | time: 0.002s
| Adam | epoch: 005 | loss: 1.60268 - acc: 0.9860 -- iter: 1/1
--
Training Step: 6  | total loss: 1.60253 | time: 0.005s
| Adam | epoch: 006 | loss: 1.60253 - acc: 0.9950 -- iter: 1/1
--
Training Step: 7  | total loss: 1.60138 | time